In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# read dataset
file_name = '/content/drive/MyDrive/data.txt'


def read_data(filename: str) -> tuple:
    data = np.loadtxt(filename, delimiter=',')
    x = data[:, 1:]
    y = data[:, 0]

    return (x, y)

In [ ]:
# dataset shape
X, y = read_data(file_name)

print(f'X:shape = {X.shape}, y:shape = {y.shape}')

In [ ]:
# count unique classes
num_class = len(np.unique(y))

print(f'Number of classes: {num_class}')

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42, shuffle=True)

In [ ]:
# encoding class labels
encoder = OneHotEncoder(categories='auto')

encoder.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
Y_train = encoder.transform(y_train.reshape(-1, 1)).toarray()
Y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

print(f'Y_train:shape = {Y_train.shape}, Y_test:shape = {Y_test.shape}')
print(f'X_train:shape = {X_train.shape}, X_test:shape = {X_test.shape}')

In [ ]:
# Multilayer Perceptron (MLP)
class MLP:
    def __init__(self, input_shape: tuple, num_class: int, verbose: bool, build: bool) -> None:
        self.input_shape = input_shape
        self.num_class = num_class
        self.verbose = verbose
        if build:
            self.model = self.build_model()

        self.model.summary()

    def build_model(self):
        input_layer = keras.layers.Input(self.input_shape)

        layer_1 = keras.layers.Dropout(0.1)(input_layer)
        layer_1 = keras.layers.Dense(500, activation='relu')(layer_1)

        layer_2 = keras.layers.Dropout(0.2)(layer_1)
        layer_2 = keras.layers.Dense(500, activation='relu')(layer_2)

        layer_3 = keras.layers.Dropout(0.2)(layer_2)
        layer_3 = keras.layers.Dense(500, activation='relu')(layer_3)

        output_layer = keras.layers.Dropout(0.3)(layer_3)
        output_layer = keras.layers.Dense(self.num_class,
                                          activation='softmax')(output_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

        self.reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                           factor=0.5,
                                                           patience=200,
                                                           min_lr=0.1)

        return model

    def fit(self, num_epochs: int, x_train: tuple, y_train: tuple, x_val: tuple, y_val: tuple) -> None:
        batch_size = 1024

        history = self.model.fit(x_train, y_train, batch_size=batch_size,
                                 epochs=num_epochs, verbose=self.verbose,
                                 validation_data=(x_val, y_val),
                                 callbacks=self.reduce_lr)

        self.model.save('MLP.hdf5')

        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        keras.backend.clear_session()

    def predict(self, x: np.array) -> np.array:
        y_pred = self.model.predict(x)

        return y_pred

In [ ]:
input_shape = X_train.shape[1:]

classifier_mlp = MLP(input_shape, num_class, False, True)

In [ ]:
classifier_mlp.fit(10000, X_train, Y_train, X_test, Y_test)

In [ ]:
# Fully Convolutional Network (FCN)
class FCN:
    def __init__(self, input_shape: tuple, num_class: int, verbose: bool, build: bool) -> None:
        self.input_shape = input_shape
        self.num_class = num_class
        self.verbose = verbose
        if build:
            self.model = self.build_model()

        self.model.summary()

    def build_model(self):
        input_layer = keras.layers.Input(self.input_shape)

        conv1 = keras.layers.Conv1D(
            filters=64, kernel_size=3, padding="same")(input_layer)
        conv1 = keras.layers.BatchNormalization()(conv1)
        conv1 = keras.layers.ReLU()(conv1)

        conv2 = keras.layers.Conv1D(
            filters=64, kernel_size=3, padding="same")(conv1)
        conv2 = keras.layers.BatchNormalization()(conv2)
        conv2 = keras.layers.ReLU()(conv2)

        conv3 = keras.layers.Conv1D(
            filters=64, kernel_size=3, padding="same")(conv2)
        conv3 = keras.layers.BatchNormalization()(conv3)
        conv3 = keras.layers.ReLU()(conv3)

        gap = keras.layers.GlobalAveragePooling1D()(conv3)

        output_layer = keras.layers.Dense(num_class, activation="softmax")(gap)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

        self.reduce_lr = keras.callbacks.ReduceLROnPlateau(
            monitor='loss', factor=0.5, patience=200, min_lr=0.1)

        return model

    def fit(self, num_epochs: int, x_train: tuple, y_train: tuple, x_val: tuple, y_val: tuple) -> None:
        batch_size = 1024

        history = self.model.fit(x_train, y_train, batch_size=batch_size,
                                 epochs=num_epochs, verbose=self.verbose,
                                 validation_data=(x_val, y_val),
                                 callbacks=self.reduce_lr)

        self.model.save('FCN.hdf5')

        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        keras.backend.clear_session()

    def predict(self, x: np.array) -> np.array:
        y_pred = self.model.predict(x)

        return y_pred

In [ ]:
# add a dimension to make it multivariate
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
input_shape = X_train.shape[1:]
classifier_fcn = FCN(input_shape, num_class, False, True)

In [ ]:
classifier_fcn.fit(1000, X_train, Y_train, X_test, Y_test)